In [96]:
import warnings
warnings.filterwarnings("ignore")

In [97]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
import re
import numpy as np

In [98]:
data = pd.read_csv('Kosalge.csv')
data

,CoachID,Name,NFL Coach Experience (thru 2023),College/Other Exp (thru 2023),Date of Birth,NFL Playing Exp,"Highest Pos NFL (1=HC, 2=Coord, 3=ST, Pos or Lower)",Highest Pos College / other Pro,# seasons as NFL Coordinator,Off/Def side of ball,1st Season as NFL HC,Tier
0,206280,"Ragone, Dave",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,483160,"Campbell, Dan",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,288004,"Glenn, Aaron",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,610308,"Johnson, Brian",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,174517,"Getsy, Luke",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...
275,953748,"Campanile, Anthony",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
276,458562,"Escalante, Alonso",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
277,323769,"Gilbert, Tony",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
278,255720,"Kuhr, Zak",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2


### Wikipedia Processing

In [99]:
coaches = data['Name'].values.tolist()
coaches = [coach.replace('Jr.,','') for coach in coaches]
ids = data['CoachID'].values.tolist()

special_cases = {'Michael Jr.': 'Mick',
                 'Mark "Bo"': 'Bo',
                 'D.J.': 'DJ',
                 'Van Gorder': 'VanGorder'}

for i in range(len(coaches)):
    coach = coaches[i]
    for special_case in special_cases:
        if special_case in coach:
            #print('True')
            coaches[i] = coach.replace(special_case, special_cases[special_case])

#print(coaches)

coaches = [(coach.split(', ')[-1]+'_'+('_'.join(coach.split(' ')[:-1])))[:-1] for coach in coaches]
coaches = [coach.strip() for coach in coaches]
coaches = [coach.replace('.','._').replace('__','_') for coach in coaches]
for i in range(len(coaches)):
    coaches[i] = [ids[i], coaches[i]]
#coaches

In [100]:
valid_counter = 0
URLs = []
suffixes = ['_(American_football_coach)', '_(American_football)']
for coach in coaches:
    URL = 'https://en.wikipedia.org/wiki/'+coach[1]
    if requests.get(URL).status_code == 200:
        valid_counter += 1
        URLs.append([coach[0], URL])
    else:
        for suffix in suffixes:
            if requests.get(URL+suffix).status_code == 200:
                valid_counter += 1
                URLs.append([coach[0], URL+suffix])
                continue

print('Number of Valid URLs: ' + str(valid_counter))
print('% of Valid URLs: ' + str(valid_counter/len(coaches)))

Number of Valid URLs: 197
% of Valid URLs: 0.7035714285714286


In [101]:
for i in range(len(URLs)):
    URL = URLs[i][1]
    for suffix in suffixes:
        if requests.get(URL+suffix).status_code == 200:
            URLs[i][1] = URL+suffix
            break

URLs

[[206280, 'https://en.wikipedia.org/wiki/Dave_Ragone'],
 [483160, 'https://en.wikipedia.org/wiki/Dan_Campbell'],
 [288004, 'https://en.wikipedia.org/wiki/Aaron_Glenn'],
 [610308,
  'https://en.wikipedia.org/wiki/Brian_Johnson_(American_football_coach)'],
 [174517, 'https://en.wikipedia.org/wiki/Luke_Getsy'],
 [995605, 'https://en.wikipedia.org/wiki/Shane_Bowen_(American_football)'],
 [373812, 'https://en.wikipedia.org/wiki/Joe_Brady_(American_football_coach)'],
 [283478, 'https://en.wikipedia.org/wiki/Mick_Lombardi'],
 [370429, 'https://en.wikipedia.org/wiki/Renaldo_Hill'],
 [780072,
  'https://en.wikipedia.org/wiki/Scott_Turner_(American_football_coach)'],
 [882423, 'https://en.wikipedia.org/wiki/Brian_VanGorder'],
 [403379, 'https://en.wikipedia.org/wiki/Adam_Stenavich'],
 [425989, 'https://en.wikipedia.org/wiki/Jonathan_Gannon'],
 [958787, 'https://en.wikipedia.org/wiki/Tim_Kelly_(American_football)'],
 [726241,
  'https://en.wikipedia.org/wiki/Ronnie_Jones_(American_football_coach)

In [102]:
def get_raw_info(URL):
    response = requests.get(URL)
    soup = BeautifulSoup(response.text, 'html')
    infobox = soup.find('table', {'class': 'infobox'})
    if type(infobox) == type(None):
        return
    text = []
    trs = infobox.find_all('tr')
    for tr in trs:
        text.append(tr.text)
    return text

In [103]:
raw_infos = []
for URL in URLs:
    #print(URL[1])
    raw_info = get_raw_info(URL[1])
    if raw_info == None:
        continue
    for row in raw_info:
        if 'coach' in row.lower():
            raw_infos.append([URL[0], raw_info])
            break

#for raw_info in raw_infos:
#    print(raw_info[0])
#print(len(raw_infos))

In [104]:
birthdates = []
for raw_info in raw_infos:
    for row in raw_info[1]:
        if 'born' in row.lower():
            #print(row)
            long_date = '-'.join(re.findall(r'\d+', row))
            birthdates.append([raw_info[0], '-'.join(long_date.split('-')[:-2][:3])])
for i in range(len(birthdates)):
    birthdate = birthdates[i]
    birthdates[i] = [birthdate[0], ('/'.join(birthdate[1].split('-')[1:]).lstrip('0').lstrip('/').replace('/0','/') + '/' + birthdate[1].split('-')[0]).lstrip('/')]

#for birthdate in birthdates:
#    print(birthdate)


#print(len(birthdates))

In [105]:
for birthdate in birthdates:
    id_, bd = birthdate[0], birthdate[1]
    data.loc[data.CoachID == id_, 'Date of Birth'] = bd

In [106]:
coach_infos = []

for raw_info in raw_infos:
    enable = False
    for i in range(len(raw_info[1])):
        row = raw_info[1][i]
        if 'as a coach' in row.lower():
            coach_infos.append([raw_info[0], raw_info[1][i+1].split('\n')])
            break

#for coach_info in coach_infos:
#    coach_info[1] = [row for row in coach_info[1] if row]
#coach_infos[0]

In [107]:
for coach_info in coach_infos:
    sections_total = []
    for row in coach_info[1]:
        sections = []
        for i in range(len(row)):
            if row[i] == '(' and ord(row[i+1]) >= 48 and ord(row[i+1]) <= 57:
                sections.append(row[:i].strip())
                start_index = i+1
            if row[i] == ')' and ((ord(row[i-1]) >= 48 and ord(row[i-1]) <= 57) or row[i-1] == 't'):
                sections.append(row[start_index:i].strip())
                sections.append(row[i+1:].strip())
        sections_total.append(sections)
    coach_info[1] = sections_total

#for coach_info in coach_infos:
#    for row in coach_info[1]:
#        print(row)
#    print()

In [108]:
def get_year_list(years):
    if '–' not in years:
        year_list = [int(years)]
        return year_list
    if 'present' in years.lower():
        year_list = list(range(int(years.split('–')[0]), 2025))
    else:
        year_list = list(range(int(years.split('–')[0]), int(years.split('–')[1])+1))

    return year_list


for coach_info in coach_infos:
    for row in coach_info[1]:
        if len(row) != 3:
            continue
        years = row[1].replace('−', '–').replace('-', '–')
        if ',' in years:
            years_sects = years.split(', ')
            year_list = []
            for sect in years_sects:
                year_list = year_list + get_year_list(sect)
        else:
            year_list = get_year_list(years)

        row[1] = year_list

#for coach_info in coach_infos:
#    for row in coach_info[1]:
#        print(row)
#    print()

In [109]:
nfl_teams = pd.read_csv('nfl_teams.csv')
nfl_teams

,Name,Abbreviation,Conference,Division
0,Arizona Cardinals,ARI,NFC,West
1,Atlanta Falcons,ATL,NFC,South
2,Baltimore Ravens,BAL,AFC,North
3,Buffalo Bills,BUF,AFC,East
4,Carolina Panthers,CAR,NFC,South
5,Chicago Bears,CHI,NFC,North
6,Cincinnati Bengals,CIN,AFC,North
7,Cleveland Browns,CLE,AFC,North
8,Dallas Cowboys,DAL,NFC,East
9,Denver Broncos,DEN,AFC,West


In [110]:
teams = nfl_teams['Name'].values.tolist()
coach_nfl_exps, coach_other_exps = [], []
for coach_info in coach_infos:
    coach_nfl_exp = []
    coach_other_exp = []
    nfl_years = []
    other_years = []
    for row in coach_info[1]:
        if len(row) != 3:
            continue
        for i in range(len(teams)):
            team = teams[i]
            if team in row[0]:
                nfl_years = nfl_years + row[1]
                coach_nfl_exp.append(row)
                break
        
            if i == len(teams)-1:
                other_years = other_years + row[1]
                coach_other_exp.append(row)
    coach_nfl_exps.append([coach_info[0], coach_nfl_exp])
    coach_other_exps.append([coach_info[0], coach_other_exp])
    if 2024 in nfl_years:
        nfl_years.remove(2024)
    if 2024 in other_years:
        other_years.remove(2024)
    coach_info.append(len(list(set(nfl_years))))
    coach_info.append(len(list(set(other_years))))

#for coach_info in coach_infos:
#    print(coach_info[0], coach_info[-1], coach_info[-2])

In [111]:
for coach_info in coach_infos:
    id_, nfl_yrs, other_yrs = coach_info[0], coach_info[-2], coach_info[-1]
    data.loc[data.CoachID == id_, 'NFL Coach Experience (thru 2023)'] = int(nfl_yrs)
    data.loc[data.CoachID == id_, 'College/Other Exp (thru 2023)'] = int(other_yrs)

In [112]:
player_infos = []

for raw_info in raw_infos:
    enable = False
    for i in range(len(raw_info[1])):
        row = raw_info[1][i]
        if 'as a player' in row.lower():
            player_infos.append([raw_info[0], raw_info[1][i+1].split('\n')])
            break
        if i == len(raw_info[1])-1:
            player_infos.append([raw_info[0], ''])

for player_info in player_infos:
    player_info[1] = [row for row in player_info[1] if row]

#for player_info in player_infos:
#    print(player_info)
#    print()

In [113]:
for player_info in player_infos:
    playing_years = []
    for team in player_info[1]:
        team_name, years = team.split(' (')[0], team.split(' (')[1]
        if years[-1] == '*': # We are not accounting for Offseason and practice squad seasons.
            continue
        if team_name not in teams:
            continue
        years = years[:-1].replace('−', '–')
        if '–' in years:
            years = list(range(int(years.split('–')[0]), int(years.split('–')[1])+1))
        else:
            years = [int(years)]
        playing_years = playing_years + years
    player_info.append(len(list(set(playing_years))))

In [114]:
for player_info in player_infos:
    id_, playing_yrs = player_info[0], player_info[-1]
    data.loc[data.CoachID == id_, 'NFL Playing Exp'] = int(playing_yrs)

In [115]:
for coach_nfl_exp in coach_nfl_exps:
    coach_positions = [exp[-1] for exp in coach_nfl_exp[1]]
    if 'Head coach' in coach_positions or 'Head Coach' in coach_positions:
        coach_nfl_exp.append(1)
        continue
    if 'Offensive coordinator' in coach_positions or 'Defensive coordinator' in coach_positions:
        coach_nfl_exp.append(2)
        continue
    coach_nfl_exp.append(3)

#for coach_nfl_exp in coach_nfl_exps:
#    print(coach_nfl_exp[0], coach_nfl_exp[-1])

In [116]:
for coach_nfl_exp in coach_nfl_exps:
    id_, exp = coach_nfl_exp[0], coach_nfl_exp[-1]
    data.loc[data.CoachID == id_, 'Highest Pos NFL (1=HC, 2=Coord, 3=ST, Pos or Lower)'] = int(exp)

In [117]:
for coach_nfl_exp in coach_nfl_exps:
    coord_years = []
    for row in coach_nfl_exp[1]:
        if 'Offensive coordinator' in row[-1] or 'Defensive coordinator' in row[-1]:
            coord_years = coord_years + row[1]
    if 2024 in coord_years:
        coord_years.remove(2024)
    coach_nfl_exp.append(len(list(set(coord_years))))

In [118]:
for coach_nfl_exp in coach_nfl_exps:
    id_, coord_yrs = coach_nfl_exp[0], coach_nfl_exp[-1]
    data.loc[data.CoachID == id_, '# seasons as NFL Coordinator'] = int(coord_yrs)

In [119]:
for coach_nfl_exp in coach_nfl_exps:
    id_ = coach_nfl_exp[0]
    first_year_hc = np.nan
    for row in coach_nfl_exp[1]:
        if 'Head coach' == row[-1] or 'head coach' == row[-1] or 'Interim head coach' == row[-1]:
            first_year_hc = int(row[1][0])
            break
    data.loc[data.CoachID == id_, '1st Season as NFL HC'] = first_year_hc

In [120]:
pos_dict = {'head coach': ['HC','Off',1],
            'quarterbacks': ['QB','Off',3],
            'offensive coordinator': ['OC','Off',2],
            'defensive coordinator': ['DC','Def',2],
            'graduate assistant': ['GA','X',4],
            'wide receivers': ['WR','Off',3],
            'student assistant': ['GA','X',4],
            'linebackers': ['LB','Def',3],
            'offensive line': ['OL','Off',3],
            'defensive line': ['DL','Dff',3],
            'special teams': ['ST','ST',3],
            'special teams coordinator': ['ST','ST',2],
            'defensive backs': ['DB','Def',3],
            'safeties': ['DB','Def',3],
            'running backs': ['RB','Off',3],
            'tight ends': ['TE','Off',3],
            'cornerbacks': ['DB','Def',3],
            'assistant head coach': ['AH','X',3],
            'assistant ': ['AP','X',3]}

In [121]:
for coach_other_exp in coach_other_exps:
    pos_history = []
    for row in coach_other_exp[1]:
        position = row[-1].lower()
        for pos in pos_dict:
            if pos in position:
                pos_history.append(pos_dict[pos])

    if len(pos_history) == 0:
        continue
    highest_pos = pos_history[-1]
    for pos in pos_history[::-1]:
        if pos[-1] < highest_pos[-1]:
            highest_pos = pos

    id_ = coach_other_exp[0]
    data.loc[data.CoachID == id_, 'Highest Pos College / other Pro'] = highest_pos[0]

In [123]:
for coach_info in coach_infos:
    sides = []
    #print(coach_info[0])
    for row in coach_info[1]:
        if len(row) != 3:
            continue
        position = row[-1].lower()
        if 'offensive' in position:
            sides.append('Off')
            continue
        if 'defensive' in position:
            sides.append('Def')
            continue
        for pos in pos_dict:
            if pos in position:
                sides.append(pos_dict[pos][1])
                break
    if len(sides) == 0:
        coach_info.append(np.nan)
    else:
        coach_info.append(max(set(sides), key = sides.count))

In [124]:
for coach_info in coach_infos:
    id_, side = coach_info[0], coach_info[-1]
    data.loc[data.CoachID == id_, 'Off/Def side of ball'] = side

### NFL.com Processing

In [189]:
data[-20:]

,CoachID,Name,NFL Coach Experience (thru 2023),College/Other Exp (thru 2023),Date of Birth,NFL Playing Exp,"Highest Pos NFL (1=HC, 2=Coord, 3=ST, Pos or Lower)",Highest Pos College / other Pro,# seasons as NFL Coordinator,Off/Def side of ball,1st Season as NFL HC,Tier
260,528773,"Newell, Gina",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
261,996932,"Jones, Dale",0.0,34.0,3/8/1963,1.0,3.0,DC,0.0,Def,NaN,2
262,556053,"Meadows, Luke",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
263,938970,"Colombo, Marc",5.0,0.0,10/8/1978,10.0,3.0,NaN,0.0,Off,NaN,2
264,989354,"Mennenga, Shawn",9.0,18.0,1/8/1971,0.0,3.0,HC,0.0,Def,NaN,2
265,542451,"Ahmad, Omar",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
266,892823,"Whitaker, Cedric",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
267,290560,"Caminiti, Chris",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
268,743484,"Udinski, Grant",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
269,403320,"Meidt, Chris",NaN,NaN,6/1/1969,0.0,NaN,NaN,NaN,NaN,NaN,2


In [129]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'
}

In [183]:
# get data from url
def get_data_text(url, headers, clean = True):
    pageTree = requests.get(url, headers=headers)
    soup = BeautifulSoup(pageTree.content, features='html.parser')
    text = [row.strip() for row in soup.get_text().split('\n') if (row.strip() != '\n' and row.strip() != '')]
    return text

In [184]:
data_text = get_data_text('https://www.giants.com/team/coaches-roster/zak-kuhr', headers=headers)

In [188]:
for i in range(len(data_text)):
    if data_text[i].lower() == 'biography':
        start_index = i
    if data_text[i].lower() == 'latest news':
        end_index = i
print('\n'.join(data_text[start_index+1:end_index]))

Kuhr (pronounced "COOR") just completed his fourth season coaching in the NFL. He was hired in 2020 as a quality control coach to work with the Titans' defense under Bowen and became the inside linebackers assistant in 2021.
Before joining the Titans, Kuhr spent the 2019 season as an assistant at the University of Texas. He also spent time at Texas State, Rutgers, James Madison, Old Dominion, Ohio State (intern), and Edward Waters University.
Zak Kuhr's Coaching Timeline:
2024-Current: New York Giants, Defensive Assistant
2021-23: Tennessee Titans, Inside Linebackers Assistant
2020: Tennessee Titans, Quality Control
2019: Texas, Assistant Coach
2018: Texas State, Assistant Head Coach/Offensive Coordinator/Running Backs
2017: Texas State, Co-Offensive Coordinator/Running Backs
2016: Rutgers, Running Backs
2015: James Madison, Co-Offensive Coordinator/Running Backs
2014: James Madison, Running Backs
2013: Old Dominion, Graduate Assistant
2011-12: Ohio State, Intern
2009-10: Edward Waters